###  Data Preprocessing Based On Embedding Coverage

When building deep learning NLP models, there are two golden rules for meaningful preprocessing:

(1) Don't use standard preprocessing steps like stemming or stopword removal when there is pre-trained embeddings. The reason is simple: You loose valuable information, which would help your NN to figure things out.

(2) Get vocabulary as close to the embeddings as possible.

In [1]:
## Load Embeddings
import pandas as pd
import numpy as np
import operator 
import re
import gc
import keras
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')

Using TensorFlow backend.


In [2]:
df = pd.read_csv('clean_data.csv')
df.shape

(735502, 4)

#### Load embeddings
I am using GloVe only, for memory usage purposes mostly.

In [3]:
def load_embed(file):
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
        
    return embeddings_index

In [4]:
glove = "glove.840B.300d.txt"
embed_glove = load_embed(glove)

#### Vocabulary and Coverage functions

In [6]:
# build vocabulary dictionary
def build_vocab(texts):
    questions = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in questions:
        for word in sentence:
            vocab[word] = vocab.get(word,1) + 1
    return vocab

# check how many words have corresponding embedding
def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys():
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.3%} of unique words(vocab)'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.3%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words

#### Starting point

In [7]:
vocab = build_vocab(df['question_text'])
print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)

Glove : 
Found embeddings for 37.220% of unique words(vocab)
Found embeddings for  86.913% of all text


64% words don't have corresponding embedding, so we need to find a way to improve the embedding coverage otherwise the embedding doesn't contribute a lot to the model.

In [10]:
oov_glove[:20]

[('vitamin_a', 12409),
 ('India?', 9799),
 ('it?', 8760),
 ("What's", 5885),
 ('do?', 5232),
 ('them?', 5119),
 ('people?', 4291),
 ('life?', 4167),
 ('world?', 4009),
 ('me?', 3800),
 ('operating_room', 3391),
 ('you?', 3375),
 ('time?', 3313),
 ('Quora?', 3165),
 ('american_samoa', 3092),
 ('why?', 3009),
 ('country?', 2657),
 ('this?', 2539),
 ('not?', 2220),
 ('women?', 2217)]

Clearly, contractions, words with punctuation don't have embedding. Let us correct that.

#### Contractions

In [12]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [13]:
def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

df['treated_question'] = df['question_text'].apply(lambda x: clean_contractions(x, contraction_mapping))

vocab = build_vocab(df['treated_question'])
print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)

Glove : 
Found embeddings for 37.375% of unique words(vocab)
Found embeddings for  87.214% of all text


#### Special Characters

In [14]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  # Other special characters that I have to deal with in last
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

df['treated_question'] = df['treated_question'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))

vocab = build_vocab(df['treated_question'])
print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)

Glove : 
Found embeddings for 81.532% of unique words(vocab)
Found embeddings for  99.429% of all text


Here we already improve the embeddings from 37% to 81%. Let's take a look at what is still wrong.

In [15]:
oov_glove[:20]

[('Quorans', 725),
 ('Brexit', 365),
 ('cryptocurrencies', 247),
 ('Redmi', 203),
 ('SJWs', 113),
 ('bhakts', 87),
 ('BNBR', 87),
 ('Bhakts', 85),
 ('Adityanath', 78),
 ('Doklam', 76),
 ('demonetisation', 69),
 ('UCEED', 65),
 ('Pokémon', 65),
 ('Blockchain', 62),
 ('faineant', 60),
 ('OnePlus', 55),
 ('Coinbase', 55),
 ('Machedo', 52),
 ('GDPR', 52),
 ('Qoura', 50)]

Current problems lie in mispells, let's manually correct them.

In [16]:
mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization', 'pokémon': 'pokemon'}

In [17]:
def correct_spelling(x, dic):
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

In [18]:
df['treated_question'] = df['treated_question'].apply(lambda x: correct_spelling(x, mispell_dict))

In [19]:
vocab = build_vocab(df['treated_question'])
print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)

Glove : 
Found embeddings for 81.535% of unique words(vocab)
Found embeddings for  99.430% of all text


In [23]:
df= df[['treated_question','target']]
df.to_csv('clean_data_after_embedding.csv',index=False)

In [24]:
df.head(5)

,treated_question,target
0,"Do you have an adopted dog , how would you en...",0
1,Why does velocity affect time ? Does velocity...,0
2,How did Otto von Guericke used the Magdeburg h...,0
3,Can I convert montra helicon D to a mountain b...,0
4,"What is the dumbest , yet possibly true expla...",0


In [25]:
len(vocab)

159078